In [34]:
%pip install pandas
%pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mannat2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.4/204.2 MB 13.5 MB/s eta 0:00:16
   ---------------------------------------- 1.2/204.2 MB 15.8 MB/s eta 0:00:13
   ---------------------------------------- 2.5/204.2 MB 19.7 MB/s eta 0:00:11
    --------------------------------------- 4.0/204.2 MB 25.7 MB/s eta 0:00:08
   - -------------------------------------- 6.5/204.2 MB 29.6 MB/s eta 0:00:07
   -- ------------------------------------- 10.6/204.2 MB 46.7 MB/s eta 0:00:05
   -- ------------------------------------- 15.1/204.2 MB 93.9 MB/s eta 0:00:03
   --- ------------------------------------ 20.3/204.2 MB 93.9 MB/s eta 0:00:02
   ---- ---------------------------------- 25.6/204.2 MB 108.8 MB/s eta 0:00:02
   ----- ---------------------------------- 29.8/204.2 MB 93.0 MB/s eta 0:00:02
   ------ --------------------------------- 34.8/204.2 MB 93.0 MB/s eta 0:00:02
   ------- -------------------------------- 39.1/204.2


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mannat2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Goal: Build a classification model to segment the customers based on the account data.

classify the Accounts eligible for Credit Line Increase without Risk, Accounts eligible for Credit Line
Increase but has risk of potential defaults or fraud, No Credit Line Increase required and Non-Performing
accounts that pose a high risk.

# Data Cleaning and Preprocessing

In [100]:
import pandas as pd

In [105]:
account_dim = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/account_dim_20250325.csv")
fraud_claim_case = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/fraud_claim_case_20250325.csv")
fraud_claim_tran = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/fraud_claim_tran_20250325.csv")
rams_batch_cur = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/rams_batch_cur_20250325.csv")
statement_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/statement_fact_20250325.csv")
syf_id = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/syf_id_20250325.csv")
transfaction_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/transaction_fact_20250325.csv")
wrld_stor_tran_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/wrld_stor_tran_fact_20250325.csv")

In [108]:
rams_batch_cur = rams_batch_cur[['cu_account_nbr', 'cu_bhv_scr', 'cu_crd_bureau_scr', 'ca_avg_utilz_lst_6_mnths', 'ca_avg_utilz_lst_6_mnths', 'mo_tot_sales_array_1', 'mo_tot_sales_array_2', 'mo_tot_sales_array_3', 'mo_tot_sales_array_4', 'mo_tot_sales_array_5', 'mo_tot_sales_array_6', 'cu_cash_line_am', 'cu_crd_line', 'cu_otb', 'cu_nbr_days_dlq', 'ca_max_dlq_lst_6_mnths', 'ca_nsf_count_lst_12_months']]
account_dim = account_dim[['current_account_nbr', 'payment_hist_1_12_mths', 'payment_hist_13_24_mths', 'overlimit_type_flag', 'ext_status_reason_cd_desc']]
statement_fact = statement_fact[['current_account_nbr', 'return_check_cnt_total']]
fraud_claim_case = fraud_claim_case[['current_account_nbr', 'gross_fraud_amt', 'net_fraud_amt']]
transfaction_fact = transfaction_fact[['current_account_nbr', 'transaction_return_cnt', 'first_purchase_ind']]

In [ ]:
rams_batch_cur = rams_batch_cur.rename(columns={"cu_account_nbr": "current_account_nbr"})
fraud_claim_case = fraud_claim_case.rename(columns={"current_account_nbr_pty": "current_account_nbr"})
master_df = account_dim.merge(rams_batch_cur, on='current_account_nbr').merge(statement_fact, on="current_account_nbr").merge(fraud_claim_case, on="current_account_nbr").merge(transfaction_fact, on="current_ac")

In [119]:
master_df

,current_account_nbr,payment_hist_1_12_mths,payment_hist_13_24_mths,overlimit_type_flag,ext_status_reason_cd_desc,cu_bhv_scr,cu_crd_bureau_scr,ca_avg_utilz_lst_6_mnths,ca_avg_utilz_lst_6_mnths,ca_avg_utilz_lst_6_mnths,...,cu_crd_line,cu_otb,cu_nbr_days_dlq,ca_max_dlq_lst_6_mnths,ca_nsf_count_lst_12_months,return_check_cnt_total,gross_fraud_amt,net_fraud_amt,transaction_return_cnt,first_purchase_ind
0,nTgalbpd5Rsol7CA,ZZZZZZIAZZZZ,ZZZZIIIQIIII,0,NaN,742,813,0,0,0,...,12900,12900.00,0,0,0,0,557.29,0.0,1,N
1,nTgalbpd5Rsol7CA,ZZZZZZIAZZZZ,ZZZZIIIQIIII,0,NaN,742,813,0,0,0,...,12900,12900.00,0,0,0,0,557.29,0.0,0,N
2,nTgalbpd5Rsol7CA,ZZZZZZIAZZZZ,ZZZZIIIQIIII,0,NaN,742,813,0,0,0,...,12900,12900.00,0,0,0,0,557.29,0.0,0,N
3,nTgalbpd5Rsol7CA,ZZZZZZIAZZZZ,ZZZZIIIQIIII,0,NaN,742,813,0,0,0,...,12900,12900.00,0,0,0,0,557.29,0.0,0,N
4,nTgalbpd5Rsol7CA,ZZZZZZIAZZZZ,ZZZZIIIQIIII,0,NaN,742,813,0,0,0,...,12900,12900.00,0,0,0,0,557.29,0.0,1,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91929,3hJcpXnkhUx7DBjb,ZZZZZZZZZZZZ,ZZZZZZZZZZZZ,0,NaN,745,0,0,0,0,...,2500,2500.00,0,0,0,0,2000.00,0.0,0,N
91930,0ctPdTAguBbpQDMF,1A,"\\\\\\""""",0,NaN,6,757,0,0,0,...,6500,6474.26,30,1,0,0,11.87,0.0,0,Y
91931,0ctPdTAguBbpQDMF,1A,"\\\\\\""""",0,NaN,6,757,0,0,0,...,6500,6474.26,30,1,0,0,11.87,0.0,0,Y
91932,0ctPdTAguBbpQDMF,1A,"\\\\\\""""",0,NaN,0,786,0,0,0,...,6500,6488.13,0,0,0,0,11.87,0.0,0,Y


In [121]:
master_df.drop(columns=['overlimit_type_flag', 'ca_avg_utilz_lst_6_mnths'], inplace=True)

In [128]:
master_df

,current_account_nbr,payment_hist_1_12_mths,payment_hist_13_24_mths,ext_status_reason_cd_desc,cu_bhv_scr,cu_crd_bureau_scr,mo_tot_sales_array_1,mo_tot_sales_array_2,mo_tot_sales_array_3,mo_tot_sales_array_4,...,cu_crd_line,cu_otb,cu_nbr_days_dlq,ca_max_dlq_lst_6_mnths,ca_nsf_count_lst_12_months,return_check_cnt_total,gross_fraud_amt,net_fraud_amt,transaction_return_cnt,first_purchase_ind
0,nTgalbpd5Rsol7CA,ZZZZZZIAZZZZ,ZZZZIIIQIIII,NaN,742,813,0.00,0.00,0.0,0.0,...,12900,12900.00,0,0,0,0,557.29,0.0,1,N
1,nTgalbpd5Rsol7CA,ZZZZZZIAZZZZ,ZZZZIIIQIIII,NaN,742,813,0.00,0.00,0.0,0.0,...,12900,12900.00,0,0,0,0,557.29,0.0,0,N
2,nTgalbpd5Rsol7CA,ZZZZZZIAZZZZ,ZZZZIIIQIIII,NaN,742,813,0.00,0.00,0.0,0.0,...,12900,12900.00,0,0,0,0,557.29,0.0,0,N
3,nTgalbpd5Rsol7CA,ZZZZZZIAZZZZ,ZZZZIIIQIIII,NaN,742,813,0.00,0.00,0.0,0.0,...,12900,12900.00,0,0,0,0,557.29,0.0,0,N
4,nTgalbpd5Rsol7CA,ZZZZZZIAZZZZ,ZZZZIIIQIIII,NaN,742,813,0.00,0.00,0.0,0.0,...,12900,12900.00,0,0,0,0,557.29,0.0,1,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91929,3hJcpXnkhUx7DBjb,ZZZZZZZZZZZZ,ZZZZZZZZZZZZ,NaN,745,0,0.00,0.00,0.0,0.0,...,2500,2500.00,0,0,0,0,2000.00,0.0,0,N
91930,0ctPdTAguBbpQDMF,1A,"\\\\\\""""",NaN,6,757,0.00,11.87,0.0,0.0,...,6500,6474.26,30,1,0,0,11.87,0.0,0,Y
91931,0ctPdTAguBbpQDMF,1A,"\\\\\\""""",NaN,6,757,0.00,11.87,0.0,0.0,...,6500,6474.26,30,1,0,0,11.87,0.0,0,Y
91932,0ctPdTAguBbpQDMF,1A,"\\\\\\""""",NaN,0,786,11.87,0.00,0.0,0.0,...,6500,6488.13,0,0,0,0,11.87,0.0,0,Y
